# Homework Solutions

## Dataset
Bank Marketing dataset:
https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv


**In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not.**


In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-13 19:54:03--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.01s   

2025-10-13 19:54:03 (5.66 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



## Data preparation

* Check if the missing values are presented in the features.
* If there are missing values:
    * For caterogiral features, replace them with 'NA'
    * For numerical features, replace with with 0.0 

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [3]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [4]:
# Categorlical and Numerical features
categorical = list(df.dtypes[df.dtypes == 'object'].index)
numerical = list(df.dtypes[df.dtypes != 'object'].index)
numerical.remove('converted')
print(categorical)
print(numerical)

['lead_source', 'industry', 'employment_status', 'location']
['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']


In [5]:
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
lead_source,paid_ads,social_media,events,paid_ads,referral,events,social_media,social_media,referral,paid_ads
industry,NaN,retail,healthcare,retail,education,manufacturing,technology,NaN,healthcare,other
number_of_courses_viewed,1,1,5,2,3,1,0,5,4,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0,59904.0,51283.0,62975.0,38648.0,59866.0
employment_status,unemployed,employed,unemployed,NaN,self_employed,NaN,NaN,student,unemployed,student
location,south_america,south_america,australia,australia,europe,africa,middle_east,europe,south_america,australia
interaction_count,4,1,3,1,3,6,2,4,2,3
lead_score,0.94,0.8,0.69,0.87,0.62,0.83,0.57,0.62,0.86,0.43
converted,1,0,1,0,1,1,0,1,1,1


In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
# For categorical features, replace them with 'NA'
df[categorical] = df[categorical].fillna('NA')

In [8]:
# For numerical features, replace with with 0.0
df[numerical] = df[numerical].fillna(0.0) 

In [10]:
# Double check if we still have any missing values
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Question 1

What is the most frequent observation (mode) for the column `industry`?

- `NA`
- `technology`
- `healthcare`
- `retail`

In [11]:
df['industry'].value_counts().sort_values(ascending=False)

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

## Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `interaction_count` and `lead_score`
- `number_of_courses_viewed` and `lead_score`
- `number_of_courses_viewed` and `interaction_count`
- `annual_income` and `interaction_count`

Only consider the pairs above when answering this question.

### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value `y` is not in your dataframe.

In [12]:
cor_pairs = [
    ['interaction_count', 'lead_score'],
    ['number_of_courses_viewed', 'lead_score'],
    ['number_of_courses_viewed', 'interaction_count'],
    ['annual_income', 'interaction_count']
]

In [13]:
# Correlations
for c in cor_pairs:
    print(f'Correlation between "{c[0]}" and "{c[1]}": {df[[c[0]]]
          .corrwith(df[c[1]])
          .abs()
          .values[0]
          .round(4)}')

Correlation between "interaction_count" and "lead_score": 0.0099
Correlation between "number_of_courses_viewed" and "lead_score": 0.0049
Correlation between "number_of_courses_viewed" and "interaction_count": 0.0236
Correlation between "annual_income" and "interaction_count": 0.027


In [14]:
# Split the data into train, validation, and test sets (60%/20%/20%)
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [15]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [16]:
# Reseting index for each dataframe
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [17]:
# Save y and remove it from the dataframes
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

## Question 3

* Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?
  
- `industry`
- `location`
- `lead_source`
- `employment_status`

In [18]:
scores = {}
for c in categorical:
    scores[c] = round(mutual_info_score(y_train, df_train[c]), 2)

scores


{'lead_source': 0.04,
 'industry': 0.01,
 'employment_status': 0.01,
 'location': 0.0}

## Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.64
- 0.74
- 0.84
- 0.94

In [21]:
# Instantiating the DictVectorizer (not using sparse matrix)
dv = DictVectorizer(sparse=False)


# Training data
train_dict = df_train[categorical + numerical].to_dict(orient='records')
# Fit and transform the training data
X_train = dv.fit_transform(train_dict)


# Validation data
# Transform the validation data
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

# Test data
# Transform the test data
test_dict = df_test[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(test_dict)

In [22]:
model = LogisticRegression(
    solver='liblinear',
    C=1.0, max_iter=1000,
    random_state=42)

In [23]:
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [30]:
# Model coefficients
w0 = model.intercept_[0]
w = model.coef_[0]
w0, w

(np.float64(-0.0691472802783609),
 array([-1.77843867e-05, -1.47154423e-02,  3.39095225e-02,  2.66248432e-03,
         1.15238518e-02, -1.02527697e-01, -2.48510995e-02,  4.93604222e-02,
        -2.01258344e-02, -1.34214865e-02, -3.00232200e-03, -9.25991830e-03,
        -3.17957304e-02, -1.60513114e-02,  3.11339155e-01,  5.12012528e-02,
         2.01511698e-02, -1.20346284e-02, -1.16021521e-02, -1.15251880e-01,
         7.95303436e-02, -2.99401329e-02,  3.95843295e-03, -1.14296944e-02,
        -1.12457415e-02, -5.59987025e-03,  8.26402635e-03,  5.58598769e-03,
        -3.33967159e-02, -2.52837052e-02,  4.53752887e-01]))

In [26]:
y_pred = model.predict_proba(X_val)[:, 1]

In [27]:
converted_decision = (y_pred >= 0.5)

In [28]:
score = (y_val == converted_decision).mean()
score

np.float64(0.6996587030716723)

In [29]:
# Rounded score to 2 decimal points
print(round(score, 2))


0.7


## Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model using the same features and parameters as in Q4 (without rounding).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?

- `'industry'`
- `'employment_status'`
- `'lead_score'`

> **Note**: The difference doesn't have to be positive.

In [118]:
model = LogisticRegression(
    solver='liblinear',
    C=1.0, max_iter=1000,
    random_state=42)

In [119]:
features = categorical + numerical
features

['lead_source',
 'industry',
 'employment_status',
 'location',
 'number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [120]:
def rotating_features(features):
    feats = {}
    for f in features:
        small_features = features.copy()
        small_features.remove(f)
        feats[f] = small_features

    return feats

In [121]:
def fit_transform_data(df_train, features):
    dict_train_data = df_train[features].to_dict(orient='records')
    X_train = dv.fit_transform(dict_train_data)

    dict_val_data = df_val[features].to_dict(orient='records')
    X_val = dv.transform(dict_val_data)

    return X_train, X_val

In [105]:
def fit_transform_data(df_train, features):
    dict_train_data = df_train[features].to_dict(orient='records')
    X_train = dv.fit_transform(dict_train_data)

    return X_train

In [122]:
def predict(model, features):
    X_train_elim, X_val_elim = fit_transform_data(df_train, features)
    model.fit(X_train_elim, y_train)
    y_pred = model.predict_proba(X_val_elim)[:, 1]
    converted_decision = (y_pred >= 0.5)
    score = (y_val == converted_decision).mean()
    return score

In [104]:
def predict(model, features):
    X_train_elim = fit_transform_data(df_train, features)
    model.fit(X_train_elim, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    score = (y_val == converted_decision).mean()
    return score

In [134]:
features_dict = rotating_features(features)

In [ ]:
scores = {}
for f in features_dict:
    feat_score = predict(model, features_dict[f])
    diff_score = abs(feat_score - score).round(4)
    scores['feature'] = f
    scores['score'] = feat_score
    scores['diff'] = diff_score
    # print(f"Without {scores['feature']}: {scores['score']}, diff: {scores['diff']}")


Without lead_source: 0.7030716723549488, diff: 0.0034
Without industry: 0.6996587030716723, diff: 0.0
Without employment_status: 0.6962457337883959, diff: 0.0034
Without location: 0.7098976109215017, diff: 0.0102
Without number_of_courses_viewed: 0.5563139931740614, diff: 0.1433
Without annual_income: 0.8532423208191127, diff: 0.1536
Without interaction_count: 0.5563139931740614, diff: 0.1433
Without lead_score: 0.7064846416382252, diff: 0.0068


In [131]:
scores

{'feature': 'lead_score',
 'score': np.float64(0.7064846416382252),
 'diff': np.float64(0.0068)}

## Question 6

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

> **Note**: If there are multiple options, select the smallest `C`.

In [ ]:
GIT_COMMITTER_DATE="2025-10-12 14:34:19" git commit --amend --no-edit --date="2025-10-12 14:34:19"